# A Closed Form Reduced Order Electrochemical Model for Lithium-Ion Cells

In [1]:
import LiionBatteryModels.AshwiniModel as am 
using Gadfly

Base.active_project()

In [ ]:
I = 13.5
t = Float64.(0:1:25000)
tspan = 10

In [ ]:
# Start model solving
dₚ₁ᵢ = am.dₚ₁₀(I)
dₙ₁ᵢ = am.dₙ₁₀(I)

In [ ]:
# Reaction rates @x = 0, x = L
jₙ = am.jₙ(0, dₙ₁ᵢ, dₚ₁ᵢ, I)
jₚ = am.jₚ(0, dₙ₁ᵢ, dₚ₁ᵢ, I)

In [ ]:
# Storing required time series of variables
# c₂ᵢₙ = zeros(size(t)[1])
# c₂ᵢₚ = zeros(size(t)[1])

c₂̄ₙ = zeros(size(t)[1])
c₂̄ₚ = zeros(size(t)[1])
c₂̄ₛ = zeros(size(t)[1])

c₂ₙ₀ = zeros(size(t)[1])
c₂ₚₗ = zeros(size(t)[1])

jₙ  = zeros(size(t)[1])
jₚ = zeros(size(t)[1]);

In [ ]:
c₂̄ₙ[1] = am.c₂₀
c₂̄ₚ[1] = am.c₂₀
c₂̄ₛ[1] = am.c₂₀

c₂ₙ₀[1] = am.c₂₀
c₂ₚₗ[1] = am.c₂₀

jₙ[1] = am.jₙ(0, dₙ₁ᵢ, dₚ₁ᵢ, I)
jₚ[1] = am.jₚ(am.L, dₙ₁ᵢ, dₚ₁ᵢ, I)

In [ ]:
for i = 2:4
    params = am.param_solver(I, tspan, c₂̄ₙ[i-1], c₂̄ₚ[i-1], c₂̄ₛ[i-1], c₂ₙ₀[i-1], c₂ₚₗ[i-1], jₙ[i-1], jₚ[i-1])

    n0 = params[1]; n2 = params[3]; n3 = params[4];
    p0 = params[5]; p2 = params[7]; p3 = params[8];
    s0 = params[9]; s2 = params[10]; s3 = params[11];

    c₂ₙ₀[i] = params[15];
    c₂ₚₗ[i] = params[16];

    println(params)
end